# Imports

In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pycaret.classification import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier as et
import warnings
warnings.filterwarnings("ignore")

# Load

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-feb-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-feb-2022/test.csv')
df = df.drop(['row_id'], axis=1).copy()
test = test.drop(['row_id'], axis=1).copy()

y = df['target'].copy()
X = df.drop('target', axis=1)


In [ ]:
print('df Shape: {}\nMissing Data: {}\nDuplicates: {}\n'\
      .format(df.shape, df.isna().sum().sum(), df.duplicated().sum()))
print('Test Shape: {}\nMissing Data: {}\nDuplicates: {}\n'\
      .format(test.shape, test.isna().sum().sum(), test.duplicated().sum()))

# Analysis

In [ ]:
# A tarefa é classificar 10 espécies diferentes de bactérias usando os dados disponibilizados.

# As bactérias são as seguintes:
# Bacteroides Fragilis - estão frequentemente associadas a doenças no trato gastrintestinal.
# Campylobacter Jejuni - causa inflamação do cólon (colite) que resulta em febre e diarreia.
# Enterococcus Hirae - normalmente causam infecções do trato urinário, infecções pélvicas e intra-abdominais, dentre outras.
# Escherichia coli - pode provocar doenças, como infecções urinárias, diarreia e a colite hemorrágica e síndrome hemolítico-urêmica.
# Escherichia Fergusonii - pode provocar abortamento, vasculite e mamite.
# Klebsiella Pneumoniae - pode produzir infecções graves, como pneumonia ou meningite.
# Salmonella Enterica - pode causar dois tipos de doença, dependendo do sorotipo: salmonelose não tifóide e febre tifoide.
# Staphylococcus Aureus - causa muitos tipos de infecção, desde furúnculos até septicemias (sepse), endocardites (infecções no coração) e abscessos.
# Streptococcus Pneumoniae - pneumonia, meningite e otite (inflamação aguda do ouvido) são exemplos de doenças que podem ser causadas por essa bactéria.
# Streptococcus Pyogenes - pode provocar doenças mais graves como a escarlatina e a febre reumática.

In [ ]:
df.head(1)

In [ ]:
df.groupby('target').describe()

In [ ]:
df.shape

In [ ]:
df.corr()

In [ ]:
for i in df.target.unique():
    cor_df=df[df.target==i]
    cor=cor_df.corr()  
    c = cor.abs().unstack().drop_duplicates().reset_index()
    c = c.rename(columns={'level_0': 'Gene 1', 'level_1': 'Gene 2', 0: 'Correlation'})
    c = c.query('Correlation < 1').sort_values(by = 'Correlation', ascending = False).reset_index(drop=True)
    print()
    display(c.iloc[:1,:].style.background_gradient(cmap='flare').set_caption('Most Correlated Genes in {}'.format(i.replace('_', ' '))))

# Training

## Setup PyCaret

In [ ]:
clf1 = setup(df, target='target', use_gpu = True, silent=True)

In [ ]:
best = compare_models(include = ['rf', 'et', 'dt'])

In [ ]:
print(best)

## Extra Trees

In [ ]:
model_et = et(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
                     oob_score=False, random_state=8018, verbose=0,
                     warm_start=False)

# 0.97073 sem ids

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model_et.fit( X,y )

In [ ]:
pred = model_et.predict(test)

In [ ]:
pred

# Submission

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')
# submission['target'] = pred
submission['target'] = pred
submission.to_csv("submit.csv", index=False)
submission

In [ ]:
# Também tentei outros parâmetros no modelo de Extra Trees. 
# Os parâmetros do código foram os melhores que consegui.
# Fiz um modelo com Random Forest que apresentou um resultado próximo a este mas este modelo ainda foi melhor.

In [ ]:
# Tem alguma dica de estudo? Sugestão de soluções? Vou ficar feliz com seu feedback!